In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_parquet("/mnt/storage_dimm2/kaggle_output/icecube-neutrinos-in-deep-ice/20230405-063040/GPS/fold_0/oofs.parquet")
df

,azimuth,zenith,azimuth_gt,zenith_gt
0,1.547212,0.525802,0.417742,1.549686
1,-2.157183,2.254038,5.252108,0.672366
2,2.022952,2.956379,1.547973,1.420000
3,0.274987,3.025253,0.922070,0.671028
4,0.651505,0.730273,1.719300,0.579796
...,...,...,...,...
1320955,1.784120,1.742544,2.158483,0.585173
1320956,-0.060911,1.754544,0.755720,2.185338
1320957,2.201704,2.856128,2.524276,1.371455
1320958,-1.254470,1.063235,5.061414,1.020417


In [3]:
def angular_dist_score_numpy(az_true, zen_true, az_pred, zen_pred):
    """
    calculate the MAE of the angular distance between two directions.
    The two vectors are first converted to cartesian unit vectors,
    and then their scalar product is computed, which is equal to
    the cosine of the angle between the two vectors. The inverse
    cosine (arccos) thereof is then the angle between the two input vectors

    Parameters:
    -----------

    az_true : float (or array thereof)
        true azimuth value(s) in radian
    zen_true : float (or array thereof)
        true zenith value(s) in radian
    az_pred : float (or array thereof)
        predicted azimuth value(s) in radian
    zen_pred : float (or array thereof)
        predicted zenith value(s) in radian

    Returns:
    --------

    dist : float
        mean over the angular distance(s) in radian
    """

    if not (
        np.all(np.isfinite(az_true))
        and np.all(np.isfinite(zen_true))
        and np.all(np.isfinite(az_pred))
        and np.all(np.isfinite(zen_pred))
    ):
        raise ValueError("All arguments must be finite")

    # pre-compute all sine and cosine values
    sa1 = np.sin(az_true)
    ca1 = np.cos(az_true)
    sz1 = np.sin(zen_true)
    cz1 = np.cos(zen_true)

    sa2 = np.sin(az_pred)
    ca2 = np.cos(az_pred)
    sz2 = np.sin(zen_pred)
    cz2 = np.cos(zen_pred)

    # scalar product of the two cartesian vectors (x = sz*ca, y = sz*sa, z = cz)
    scalar_prod = sz1 * sz2 * (ca1 * ca2 + sa1 * sa2) + (cz1 * cz2)

    # scalar product of two unit vectors is always between -1 and 1, this is against nummerical instability
    # that might otherwise occure from the finite precision of the sine and cosine functions
    scalar_prod = np.clip(scalar_prod, -1, 1)

    # convert back to an angle (in radian)
    return np.average(np.abs(np.arccos(scalar_prod)))

In [4]:
df.describe()

,azimuth,zenith,azimuth_gt,zenith_gt
count,1.320960e+06,1.320960e+06,1.320960e+06,1.320960e+06
mean,-3.688147e-02,1.541196e+00,3.143624e+00,1.534018e+00
std,1.809538e+00,8.842301e-01,1.813718e+00,6.900986e-01
min,-3.141592e+00,6.075860e-04,4.262943e-06,7.469018e-04
25%,-1.609421e+00,8.677852e-01,1.574720e+00,1.004851e+00
50%,-2.202656e-02,1.477861e+00,3.141234e+00,1.526694e+00
75%,1.512612e+00,2.272427e+00,4.717242e+00,2.054213e+00
max,3.141582e+00,3.137478e+00,6.283173e+00,3.139704e+00


In [5]:
az_pred, zen_pred, az_true, zen_true = df["azimuth"], df["zenith"], df["azimuth_gt"], df["zenith_gt"]


angular_dist_score_numpy(az_true, zen_true, az_pred, zen_pred)

0.9894511

In [6]:
az_pred = np.where(az_pred < 0, az_pred + 2 * np.pi, az_pred)

angular_dist_score_numpy(az_true, zen_true, az_pred, zen_pred)

0.9894510566785831

In [7]:
az_pred.min(), az_pred.max()

(1.6820938981254585e-05, 6.283184351295766)